In [1]:
import os

if not os.path.exists('data'):
    !mkdir data/
if not os.path.exists('data/data.zip'):
    !wget -O data/data.zip https://disk.yandex.ru/d/l6SZZDErQ5HXkQ
if os.path.exists('data/data.zip'):
    !unzip data/data.zip -d data
    !rm data/data.zip 

Archive:  data/data.zip
   creating: data/texts/
  inflating: data/texts/20091231boeviks.txt  
  inflating: data/texts/20091231kids.txt  
  inflating: data/texts/20091231postpone.txt  
  inflating: data/texts/20091231silvio.txt  
  inflating: data/texts/20100101aes.txt  
  inflating: data/texts/20100101celebrate.txt  
  inflating: data/texts/20100101charges.txt  
  inflating: data/texts/20100101degas.txt  
  inflating: data/texts/20100101dismay.txt  
  inflating: data/texts/20100101eu.txt  
  inflating: data/texts/20100101fcc.txt  
  inflating: data/texts/20100101find.txt  
  inflating: data/texts/20100101firework.txt  
  inflating: data/texts/20100101free1.txt  
  inflating: data/texts/20100101french.txt  
  inflating: data/texts/20100101invite.txt  
  inflating: data/texts/20100101lamehorse.txt  
  inflating: data/texts/20100101massen.txt  
  inflating: data/texts/20100101moore.txt  
  inflating: data/texts/20100101movie.txt  
  inflating: data/texts/20100101murders.txt  
  inflating

In [2]:
import glob
text_files = glob.glob('data/texts/*.txt')
print('Всего файлов = ', len(text_files))

Всего файлов =  36446


In [3]:
alphabet = set()
text_len = 0
text = ''
for file in text_files:
    with open(file, 'r') as f:
        print(file)
        s = ' '.join(f.readlines())
        text += s + ' ' # ' '*64 # не будем захватывать текст из соседних статей? 
        text_len += len(s)
        alphabet.union(set(s))

data/texts/20091231boeviks.txt
data/texts/20091231kids.txt
data/texts/20091231postpone.txt
data/texts/20091231silvio.txt
data/texts/20100101aes.txt
data/texts/20100101celebrate.txt
data/texts/20100101charges.txt
data/texts/20100101degas.txt
data/texts/20100101dismay.txt
data/texts/20100101eu.txt
data/texts/20100101fcc.txt
data/texts/20100101find.txt
data/texts/20100101firework.txt
data/texts/20100101free1.txt
data/texts/20100101french.txt
data/texts/20100101invite.txt
data/texts/20100101lamehorse.txt
data/texts/20100101massen.txt
data/texts/20100101moore.txt
data/texts/20100101movie.txt
data/texts/20100101murders.txt
data/texts/20100101notebook.txt
data/texts/20100101ornament.txt
data/texts/20100101osce.txt
data/texts/20100101owls.txt
data/texts/20100101peace.txt
data/texts/20100101regret.txt
data/texts/20100101shutdown.txt
data/texts/20100101thebestof.txt
data/texts/20100101three.txt
data/texts/20100101tomsk.txt
data/texts/20100101urus.txt
data/texts/20100101vodka.txt
data/texts/20100

In [4]:
print(f'Всего текстов: {len(text_files)}')
print(f'Длина текстов: {text_len % 2**20} Мб')

Всего текстов: 36446
Длина текстов: 369581 Мб


In [5]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print('Размер словаря до чистки = ', vocab_size)

Размер словаря до чистки =  531


In [12]:
allowed_symbols = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyzЁАБВГДЕЖЗИЙКЛМНОПРСТУФХЦЧШЩЫЬЭЮЯабвгдежзийклмнопрстуфхцчшщъыьэюяё0123456789 .,-=" 
def clean_text(text):
    ret = ''
    for w in text:
        if w in allowed_symbols:
            ret+=w
    return ret

In [32]:
# так делаем, так как знаем что корпус маленький
text = clean_text(text)

In [33]:
chars = sorted(list(set(text).union(allowed_symbols)))
vocab_size = len(chars)
print('Размер словаря после чистки = ', vocab_size)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string


Размер словаря после чистки =  132


In [34]:
import torch 
data = torch.tensor(encode(text), dtype=torch.long)
n = len(data)
train_data = data[:int(n*0.9)]
val_data   = data[int(n*0.9):]

In [35]:
train_data

tensor([  0,  85, 113,  ...,   0, 114, 113])

In [36]:
import numpy as np
import pickle

# encode both to integers
print(f"train has {len(train_data):,} tokens")
print(f"val has {len(val_data):,} tokens")

# export to bin files
torch.save(train_data, 'data/train.pt')
torch.save(val_data, 'data/val.pt')

train has 49,538,287 tokens
val has 5,504,255 tokens


In [37]:
# save the meta information as well, to help us encode/decode later
meta = {
    'vocab_size': vocab_size,
    'itos': itos,
    'stoi': stoi,
}
with open('data/meta.pkl', 'wb') as f:
    pickle.dump(meta, f)